## 문제 11

In [1]:
# =============================================================================
# =============================================================================
# # 문제 11 유형(DataSet_11.csv 이용)

# 구분자 : comma(“,”), 470 Rows, 4 Columns, UTF-8 인코딩

# 세계 각국의 행복지수를 비롯한 여러 정보를 조사한 DS리서치는
# 취합된 자료의 현황 파악 및 간단한 통계분석을 실시하고자 한다.

# 컬 럼 / 정 의 / Type
# Country / 국가명 / String
# Happiness_Rank / 당해 행복점수 순위 / Double
# Happiness_Score / 행복점수 / Double
# year / 년도 / Double
# =============================================================================
# =============================================================================

import pandas as pd
import numpy as np

data11 = pd.read_csv('./DataSet/DataSet_11.csv')

In [17]:
# =============================================================================
# 1.분석을 위해 3년 연속 행복지수가 기록된 국가의 데이터를 사용하고자 한다. 
# 3년 연속 데이터가 기록되지 않은 국가의 개수는?
# - 국가명 표기가 한 글자라도 다른 경우 다른 국가로 처리하시오.
# - 3년 연속 데이터가 기록되지 않은 국가 데이터는 제외하고 이를 향후 분석에서
# 활용하시오.(답안 예시) 1
# =============================================================================

# 1. 국가명 표기가 한 글자라도 다른 경우 다른 국가로 처리
q11_1 = data11.groupby('Country').apply(len)

# 3년 연속 데이터가 기록되지 않은 국가 데이터
len(q11_1[q11_1 < 3])

# 답 : 20

20

In [37]:
# =============================================================================
# 2.(1번 산출물을 활용하여) 2017년 행복지수와 2015년 행복지수를 활용하여 국가별
# 행복지수 증감률을 산출하고 행복지수 증감률이 가장 높은 3개 국가를 행복지수가
# 높은 순서대로 차례대로 기술하시오.
# 증감률 = (2017년행복지수−2015년행복지수)/2
# 
# (답안 예시) Korea, Japan, China
# =============================================================================

# 1번 산출물
country_list = q11_1[q11_1 == 3].index

q11_2 = data11[data11.Country.isin(country_list)]

# 2. 2017년 행복지수와 2015년 행복지수를 활용
q11_2_tab = pd.pivot_table(q11_2, index = 'Country',
                            columns = 'year',
                            values = 'Happiness_Score')

# 3. 국가별 행복지수 증감률을 산출
q11_2_tab['ratio'] = (q11_2_tab[2017] - q11_2_tab[2015]) / 2

# 4. 복지수 증감률이 가장 높은 3개 국가를 행복지수가 높은 순서대로
q11_2_tab.ratio.nlargest(3).index

# 답 : 'Latvia', 'Romania', 'Togo'

Index(['Latvia', 'Romania', 'Togo'], dtype='object', name='Country')

In [50]:
# =============================================================================
# 3.(1번 산출물을 활용하여) 년도별 행복지수 평균이 유의미하게 차이가 나는지
# 알아보고자 한다. 
# 이와 관련하여 적절한 검정을 사용하고 검정통계량을 기술하시오.
# - 해당 검정의 검정통계량은 자유도가 2인 F 분포를 따른다.
# - 검정통계량은 소수점 넷째 자리까지 기술한다. (답안 예시) 0.1234
# =============================================================================

# (참고)
# from statsmodels.formula.api import ols
# from statsmodels.stats.anova import anova_lm

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy.stats import f_oneway

# 1. 1번 산춣물 활용
country_list = q11_1[q11_1 == 3].index
q11_3 = data11[data11.Country.isin(country_list)]

# 2. 년도별 행복지수 평균
q11_3_tab = pd.pivot_table(q11_3,
                          index = 'Country',
                          columns = 'year',
                          values = 'Happiness_Score')

q11_3_f = f_oneway(q11_3_tab[2015],q11_3_tab[2016],q11_3_tab[2017])
# F_onewayResult(statistic=0.004276725037689305, pvalue=0.9957324489944479)

# 3. anova (ols 적용)
q11_3_out = ols('Happiness_Score~C(year)', q11_3).fit()

q11_3_out.summary()
anova_lm(q11_3_out)

# 4. 검정통계량
q11_3_f.statistic  # 0.004276725037689305
anova_lm(q11_3_out)['F'][0]  # 0.004276725037677239

# 답 : 0.0042

0.004276725037677239

In [53]:
# [참고]
from statsmodels.stats.multicomp import pairwise_tukeyhsd  # 다중비교, 사후 검정
tukey_out=pairwise_tukeyhsd(q11_3['Happiness_Score'], q11_3['year'])

print(tukey_out)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
  2015   2016  -0.0112 0.9962 -0.3261 0.3038  False
  2015   2017   -0.001    1.0 -0.3159  0.314  False
  2016   2017   0.0102 0.9968 -0.3047 0.3251  False
---------------------------------------------------


## 문제 12

In [126]:
# =============================================================================
# =============================================================================
# # 문제 12 유형(DataSet_12.csv 이용)

# 구분자 : comma(“,”), 5000 Rows, 7 Columns, UTF-8 인코딩

# 직장인의 독서 실태를 분석하기 위해서 수도권 거주자 5000명을
# 대상으로 간단한 인적 사항과 연간 독서량 정보를 취합하였다.

# 컬 럼 / 정 의 / Type
# Age / 나이 / String
# Gender / 성별(M: 남성) / String
# Dependent_Count / 부양가족 수 / Double
# Education_Level / 교육 수준 / String
# is_Married / 결혼 여부(1: 결혼) / Double
# Read_Book_per_Year / 연간 독서량(권) / Double
# Income_Range / 소득 수준에 따른 구간(A < B < C < D < E)이며 X는
# 정보 누락 / String
# =============================================================================
# =============================================================================

import pandas as pd
import numpy as np

data12 = pd.read_csv('./DataSet/DataSet_12.csv')

In [127]:
# =============================================================================
# 1.수치형 변수를 대상으로 피어슨 상관분석을 실시하고 연간 독서량과 가장
# 상관관계가 강한 변수의 상관계수를 기술하시오
# - 상관계수는 반올림하여 소수점 셋째 자리까지 기술하시오. (답안 예시) 0.123
# =============================================================================

# 1.수치형 변수를 대상
x_list = list(data12.columns[data12.dtypes == 'int64'])
# ['Dependent_Count', 'is_Married', 'Read_Book_per_Year']

# 2. 피어슨 상관분석
q12_1 = data12[x_list].corr().drop('Read_Book_per_Year')['Read_Book_per_Year']

abs(q12_1).max()
# 0.1325908414916882 

# 답 : 0.797

0.7968432255640413

In [128]:
# =============================================================================
# 2.석사 이상(석사 및 박사) 여부에 따라서 연간 독서량 평균이 유의미하게 다른지 가설
# 검정을 활용하여 알아보고자 한다. 독립 2표본 t검정을 실시했을 때 
# 유의 확률(pvalue)의 값을 기술하시오.
# - 등분산 가정 하에서 검정을 실시한다.
# - 유의 확률은 반올림하여 소수점 셋째 자리까지 기술한다. (답안 예시) 0.123
# =============================================================================

q12_2 = data12.copy()

# 1. 석사 이상(석사 및 박사) 여부
q12_2['grad'] = data12.Education_Level.isin(['석사','박사'])+0

# 2. 독립 2표본 t검정 (ttest_ind) 등분산 가정
from scipy.stats import ttest_ind

q12_2_1 = q12_2[q12_2.grad == 1]['Read_Book_per_Year']
q12_2_0 = q12_2[q12_2.grad == 0]['Read_Book_per_Year']

q12_2_out = ttest_ind(q12_2_1, q12_2_0, equal_var=True)

q12_2_out.pvalue

# 답 : 0.269

0.2685589229897138

In [131]:
# =============================================================================
# 3.독서량과 다른 수치형 변수의 관계를 다중선형회귀분석을 활용하여 알아보고자 한다. 
# 연간 독서량을 종속변수, 나머지 수치형 자료를 독립변수로 한다. 이렇게 생성한
# 선형회귀 모델을 기준으로 다른 독립변수가 고정이면서 나이만 다를 때, 40살은 30살
# 보다 독서량이 얼마나 많은가?
# - 학사 이상이면서 소득 구간 정보가 있는 데이터만 사용하여 분석을 실시하시오.
# - 결과값은 반올림하여 정수로 표기하시오. (답안 예시) 1
# =============================================================================

# (참고)
# from statsmodels.formula.api import ols

from statsmodels.formula.api import ols

# 1. 학사 이상이면서 소득 구간 정보가 있는 데이터만 사용
q12_3 = data12[data12['Education_Level'].isin(['학사','석사','박사'])]
q12_3 = q12_3[~q12_3.Income_Range.isin(['X'])]

# 2. 연간 독서량을 종속변수, 나머지 수치형 자료를 독립변수
var_list = list(q12_3.columns[q12_3.dtypes == 'int64'].drop('Read_Book_per_Year'))

# 선형회귀 모델
q12_3_out = ols('Read_Book_per_Year~' + '+'.join(var_list), q12_3).fit()

q12_3_out.summary()

q12_3_out.params
# Intercept         -0.382146
# Age                0.796440
# Dependent_Count   -0.250269
# is_Married         0.049624

q12_3_out.params['Age']*(40-30)

# 답 : 8

7.964402066284841

## 문제 13

In [134]:
# =============================================================================
# =============================================================================
# # 문제 13 유형(DataSet13_train.csv / DataSet13_test.csv  이용)

# 구분자 : 
#     comma(“,”), 1500 Rows, 10 Columns, UTF-8 인코딩 / 
#     comma(“,”), 500 Rows, 10 Columns, UTF-8 인코딩

# 전국의 데이터 분석가 2000명을 대상으로 이직 관련 설문조사를 실시하였다. 
# 설문 대상자의 특성 및 이직 의사와 관련 인자를 면밀히 살펴보기 위해 다양한
# 분석을 실시하고자 한다.

# 컬 럼 / 정 의 / Type
# city_development_index / 거주 도시 개발 지수 / Double
# gender / 성별 / String
# relevent_experience / 관련 직무 경험 여부(1 : 유경험) / Integer
# enrolled_university / 대학 등록 형태(1 : 풀타임/파트타임) / Integer
# education_level / 교육 수준 / String
# major_discipline / 전공 / String
# experience / 경력 / Double
# last_new_job / 현 직장 직전 직무 공백 기간 / Double
# training_hours / 관련 직무 교육 이수 시간 / Double
# target / 이직 의사 여부(1 : 의사 있음) / Integer
# =============================================================================
# =============================================================================

import pandas as pd

data13 = pd.read_csv('./DataSet/DataSet_13_train.csv')

In [140]:
# =============================================================================
# 1.(Dataset_13_train.csv를 활용하여) 경력과 최근 이직시 공백기간의 상관관계를 보고자
# 한다. 남여별 피어슨 상관계수를 각각 산출하고 더 높은 상관계수를 기술하시오.
# - 상관계수는 반올림하여 소수점 둘째 자리까지 기술하시오. (답안 예시) 0.12
# =============================================================================
data13.columns
# ['city_development_index', 'gender', 'relevent_experience',
#        'enrolled_university', 'education_level', 'major_discipline',
#        'experience', 'last_new_job', 'training_hours', 'target']

# 1. 경력과 최근 이직시 공백기간의 상관관계
data13.groupby('gender')[['experience','last_new_job']].corr()

# Female 0.451898 > 0.411155
# 답 : 0.45

experience  last_new_job
gender                                       
Female experience      1.000000      0.451898
       last_new_job    0.451898      1.000000
Male   experience      1.000000      0.411155
       last_new_job    0.411155      1.000000

In [161]:
# =============================================================================
# 2.(Dataset_13_train.csv를 활용하여) 기존 데이터 분석 관련 직무 경험과 이직 의사가 서로
# 관련이 있는지 알아보고자 한다. 이를 위해 독립성 검정을 실시하고 해당 검정의 p-value를 기술하시오.
# - 검정은 STEM 전공자를 대상으로 한다.
# - 검정은 충분히 발달된 도시(도시 개발 지수가 제 85 백분위수 초과)에 거주하는 사람을
# 대상으로 한다.
# - 이직 의사 여부(target)은 문자열로 변경 후 사용한다.
# - p-value는 반올림하여 소수점 둘째 자리까지 기술하시오. (답안 예시) 0.12
# =============================================================================

from scipy.stats import chi2_contingency

# 1. 이직 의사 여부(target)은 문자열로 변경 
q13_2 = data13.copy()
q13_2.target = q13_2.target.astype('str')
# q13_2.info()

# 2.  STEM 전공자, 충분히 발달된 도시(도시 개발 지수가 제 85 백분위수 초과)에 거주하는 사람을 대상
q13_2['city_development_index'].value_counts()
base = q13_2['city_development_index'].quantile(0.85)

q13_2_df = q13_2[(q13_2.major_discipline == 'STEM') & (q13_2['city_development_index'] > base)]

# 3. 범주형 데이터 독립성 검정을 실시 : 카이스퀘어 검정
q13_2_tab = pd.crosstab(index=q13_2_df.relevent_experience, columns=q13_2_df.target)

chi_test = chi2_contingency(q13_2_tab)

pvalue = chi_test[1]

round(pvalue, 2)
# 답 : 0.64

0.64

In [169]:
# =============================================================================
# 3.(Dataset_13_train.csv를 활용하여) 인사팀에서는 어떤 직원이 이직 의사를 가지고 있을지
# 사전에 파악하고 1:1 면담 등 집중 케어를 하고자 한다. 이를 위해 의사결정 나무를
# 활용하여 모델을 생성하고 그 정확도를 확인하시오.
# - target을 종속변수로 하고 나머지 변수 중 String이 아닌 변수를 독립변수로 한다.
# - 학습은 전부 기본값으로 실시한다.
# - 평가는 "Dataset_13_test.csv" 데이터로 실시한다.
# - 정확도는 반올림하여 소수점 둘째 자리까지 기술하시오. (답안 예시) 0.12
# 
# =============================================================================

# (참고)
# from sklearn.tree import DecisionTreeClassifier
# random_state = 123

from sklearn.tree import DecisionTreeClassifier

test = pd.read_csv('./DataSet/DataSet_13_test.csv')

# data13.info()

# 1. target을 종속변수로 하고 나머지 변수 중 String이 아닌 변수를 독립변수로 한다.
var_list = data13.columns[data13.dtypes != 'object'].drop('target')

dt = DecisionTreeClassifier(random_state=123)
dt.fit(data13[var_list], data13['target'])

score = dt.score(test[var_list], test['target'])

print(f'답 : {round(score,2)}')
# 답 : 0.67

답 : 0.67


In [175]:
# =============================================================================
# =============================================================================
# # 문제 14 유형(DataSet_14.csv 이용)
#
# 구분자 : comma(“,”), 2000 Rows, 9 Columns, UTF-8 인코딩
#
# 온라인 교육업체 싱글캠퍼스에서 런칭한 교육 플랫폼을 보다
# 체계적으로 운영하기 위해 2014년부터 2016년 동안 개설된 강좌
# 2000개를 대상으로 강좌 실적 및 고객의 서비스 분석을 실시하려고
# 한다. 관련 데이터는 다음과 같다.
#
# 컬 럼 / 정 의 / Type
# id / 강좌 일련번호 / Double
# published / 강과 개설일 / String
# subject / 강좌 대주제 / String
# level / 난이도 / String
# price / 가격(만원) / Double
# subscribers / 구독자 수(결제 인원) / Double
# reviews / 리뷰 개수 / Double
# lectures / 강좌 영상 수 / Double
# duration / 강좌 총 길이(시간) / Double
# =============================================================================
# =============================================================================

import pandas as pd

data14 = pd.read_csv('./DataSet/DataSet_14.csv')
data14.columns

Index(['id', 'published', 'subject', 'level', 'price', 'subscribers',
       'reviews', 'lectures', 'duration'],
      dtype='object')

In [182]:
# =============================================================================
# 1.결제 금액이 1억 이상이면서 구독자의 리뷰 작성 비율이 10% 이상인 교육의 수는?
# - 결제 금액은 강좌 가격에 구독자 수를 곱한 값이다.
# - 리뷰 작성 비율은 리뷰 개수에 구독자 수를 나눈 값이다. (답안 예시) 1
# =============================================================================

q14_1 = data14.copy()

# 1. 결제 금액은 강좌 가격에 구독자 수를 곱한 값
q14_1['income'] = q14_1['price'] * q14_1['subscribers']

# 2. 리뷰 작성 비율은 리뷰 개수에 구독자 수를 나눈 값
q14_1['ratio'] = q14_1['reviews'] / q14_1['subscribers']

# 3. 결제 금액이 1억 이상이면서 구독자의 리뷰 작성 비율이 10% 이상인 교육의 수
result = len(q14_1[(q14_1.income >= 10000) & (q14_1.ratio >= 0.1)])

print(f'답 : {result}')

답 : 59


In [199]:
# =============================================================================
# 2.강좌 가격이 비쌀수록 구독자 숫자는 줄어든다는 가설을 확인하기 위해 상관분석을
# 실시하고자 한다. 2016년 개설된 Web Development 강좌를 대상으로 강좌 가격과
# 구독자 수의 피어슨 상관관계를 기술하시오.
# - 상관계수는 반올림하여 소수점 둘째 자리까지 기술하시오. (답안 예시) 0.12
# =============================================================================

q14_2 = data14.copy()

# 1. 2016년 개설된 Web Development 강좌를 대상
q14_2.published = pd.to_datetime(q14_2.published)
q14_2['year'] = q14_2.published.dt.year

q14_2_df = q14_2[(q14_2.year == 2016) & (q14_2.subject == 'Web Development')]

# 2. 강좌 가격과 구독자 수의 피어슨 상관관계
q14_2_df[['price','subscribers']].corr()

# 0.034392
# 답 : 0.03

,price,subscribers
price,1.000000,0.034392
subscribers,0.034392,1.000000


In [207]:
# =============================================================================
# 3.유저가 서비스 사용에 익숙해지고 컨텐츠의 좋은 내용을 서로 공유하려는 경향이
# 전반적으로 증가하는 추세라고 한다. 이를 위해 먼저 강좌 개설 년도별 구독자의 리뷰
# 작성 비율의 평균이 강좌 개설 년도별로 차이가 있는지 일원 분산 분석을 통해서
# 알아보고자 한다. 이 때 검정통계량을 기술하시오.
# - 검정통계량은 반올림하여 소수점 첫째 자리까지 기술하시오. (답안 예시) 0.1
#
# (참고)
# from statsmodels.formula.api import ols
# from statsmodels.stats.anova import anova_lm
# =============================================================================

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

q14_3 = data14.copy()
q14_3.published = pd.to_datetime(q14_3.published)
q14_3['year'] = q14_3.published.dt.year
q14_3['ratio'] = q14_3['reviews'] / q14_3['subscribers']

# 1. 강좌 개설 년도별 구독자의 리뷰 작성 비율의 평균
model = ols('ratio ~ C(year)', q14_3).fit()
result = anova_lm(model)

print(f'답 : {round(result.F[0], 1)}')
# 답 : 18.5

답 : 18.5


## 문제 15

In [211]:
# =============================================================================
# =============================================================================
# # 문제 05 유형(Dataset_05_Mart_POS.csv /  이용)
#
# =============================================================================
# Dataset_05_Mart_POS.csv 
# 구분자 : comma(“,”), 20488 Rows, 3 Columns, UTF-8 인코딩
# =============================================================================
#
# 원룸촌에 위치한 A마트는 데이터 분석을 통해 보다 체계적인 재고관리와
# 운영을 하고자 한다. 이를 위해 다음의 두 데이터 세트를 준비하였다.
#
# 컬 럼 / 정 의 / Type
# Member_number / 고객 고유 번호 / Double
# Date / 구매일 / String
# itemDescription / 상품명 / String

# =============================================================================
# Dataset_05_item_list.csv 
# 구분자 : comma(“,”), 167 Rows, 4 Columns, UTF-8 인코
# =============================================================================
#
# 컬 럼 / 정 의 / Type
# prod_id / 상품 고유 번호 / Double
# prod_nm / 상품명 / String
# alcohol / 주류 상품 여부(1 : 주류) / Integer
# frozen / 냉동 상품 여부(1 : 냉동) / Integer
# =============================================================================
# =============================================================================

import pandas as pd
data15_mart = pd.read_csv('./DataSet/DataSet_15_Mart_POS.csv')
data15_item = pd.read_csv('./DataSet/Dataset_15_item_list.csv')

In [221]:
# =============================================================================
# 1.(Dataset_05_Mart_POS.csv를 활용하여) 가장 많은 제품이 팔린 날짜에 가장 많이 팔린
# 제품의 판매 개수는? (답안 예시) 1
# =============================================================================

date = data15_mart.Date.value_counts().idxmax()
result = data15_mart[data15_mart.Date == date].itemDescription.value_counts().max()

print(f'답 : {result}')
# 답 : 7

답 : 7


In [253]:
# =============================================================================
# 2. (Dataset_05_Mart_POS.csv, Dataset_05_item_list.csv를 활용하여) 고객이 주류 제품을
# 구매하는 요일이 다른 요일에 비해 금요일과 토요일이 많을 것이라는 가설을 세웠다. 
# 이를 확인하기 위해 금요일과 토요일의 일별 주류제품 구매 제품 수 평균과 다른
# 요일의 일별 주류제품 구매 제품 수 평균이 서로 다른지 비교하기 위해 독립 2표본
# t검정을 실시하시오. 
# 해당 검정의 p-value를 기술하시오.
# - 1분기(1월 ~ 3월) 데이터만 사용하여 분석을 실시하시오.
# - 등분산 가정을 만족하지 않는다는 조건 하에 분석을 실시하시오.
# - p-value는 반올림하여 소수점 둘째 자리까지 기술하시오. (답안 예시) 0.12
# =============================================================================

from scipy.stats import ttest_ind

mart = data15_mart.copy()
item = data15_item.copy()

# 1. 구매하는 요일이 다른 요일에 비해 금요일과 토요일이 많을 것
mart['Date'] = pd.to_datetime(mart.Date)
mart['year'] = mart['Date'].dt.year
mart['month'] = mart['Date'].dt.month
mart['day'] = mart['Date'].dt.day_name(locale='ko_kr')
mart['date'] = np.where(mart['day'].isin(['금요일','토요일']), 1, 0)

# 2. 테이블 결합
q15_2 = pd.merge(mart, item, how='left', left_on='itemDescription', right_on='prod_nm')

# 3. 필터링 1분기(1월 ~ 3월) 데이터
q15_2_df = q15_2[q15_2['month'].isin([1,2,3])]

# 4. 금요일과 토요일의 일별 주류제품 구매 제품 수 평균과 다른
# 요일의 일별 주류제품 구매 제품 수 평균이 서로 다른지 비교하기 위해 독립 2표본 t검정

# -  일별 주류제품 구매 제품 수 
q15_2_df2 = pd.pivot_table(q15_2_df, index='Date',
                          columns='date', 
                          values='alcohol',
                          aggfunc='sum')

# 5. ttest 진행 (등분산 X)
q15_2_out = ttest_ind(q15_2_df2[1].dropna(),
                     q15_2_df2[0].dropna(),
                     equal_var=False)

# 6. pvalue
q15_2_out.pvalue
# Ttest_indResult(statistic=-2.335264239960428, pvalue=0.023062611047582393)

print(f'답 : {round(q15_2_out.pvalue,2)}')
# 답 : 0.02

답 : 0.02


In [281]:
# =============================================================================
# 3.(Dataset_05_Mart_POS.csv를 활용하여) 1년 동안 가장 많이 판매된 10개 상품을 주력
# 상품으로 설정하고 특정 요일에 프로모션을 진행할지 말지 결정하고자 한다. 먼저
# 요일을 선정하기 전에 일원 분산 분석을 통하여 요일별 주력 상품의 판매 개수의
# 평균이 유의미하게 차이가 나는지 알아보고자 한다. 이와 관련하여 일원 분산 분석을
# 실시하고 p-value를 기술하시오.
# - p-value는 반올림하여 소수점 둘째 자리까지 기술하시오. (답안 예시) 0.12
# 
# (참고)
# from statsmodels.formula.api import ols
# from statsmodels.stats.anova import anova_lm
# =============================================================================

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

mart = data15_mart.copy()

# 1. 1년 동안 가장 많이 판매된 10개 상품을 주력 상품으로 설정
item = list(mart['itemDescription'].value_counts().nlargest(10).index)

# 2. 일원 분산 분석을 통하여 요일별 주력 상품의 판매 개수의 평균
# - 요일별 주력 상품의 판매 개수
mart['Date'] = pd.to_datetime(mart.Date)
mart['year'] = mart['Date'].dt.year
mart['month'] = mart['Date'].dt.month
mart['day'] = mart['Date'].dt.day_name(locale='ko_kr')  # 요일

q15_3 = mart[mart['itemDescription'].isin(item)]

q15_3_df = q15_3.groupby(['Date','day'])['itemDescription'].apply(len).reset_index()

# pd.pivot_table(q15_3, index=['Date','day'],
#               values='itemDescription',
#               aggfunc='count').reset_index()

# 3.  평균이 유의미하게 차이가 나는지 일원분산분석
q15_3_ols = ols('itemDescription ~ day', q15_3_df).fit()
result = anova_lm(q15_3_ols)

print(f"답 : {round(result['PR(>F)'][0], 2)}")

# 답 : 0.52

답 : 0.52
